In [1]:
!pip install us

In [2]:
#Dependencies
from census import Census
from config_census import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from us import states
from scipy.stats import linregress
from matplotlib import pyplot as plt
import gmaps.datasets
# Census API Key
c = Census(census_key, year=2016)

In [3]:
c

In [4]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

census_pd.head()

,Name,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Zipcode
0,ZCTA5 05762,58558.0,513.0,39.2,24235.0,50.0,05762
1,ZCTA5 05763,54968.0,2715.0,43.9,29674.0,488.0,05763
2,ZCTA5 05764,44468.0,3370.0,41.3,22651.0,323.0,05764
3,ZCTA5 05765,56771.0,1632.0,46.7,30786.0,85.0,05765
4,ZCTA5 05766,52250.0,598.0,45.9,33227.0,76.0,05766


In [5]:
census_pd.count()

Name                 33120
Household Income     33120
Population           33120
Median Age           33120
Per Capita Income    32836
Poverty Count        33120
Zipcode              33120
dtype: int64

In [6]:
census_pd.dropna(how='any')

,Name,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Zipcode
0,ZCTA5 05762,58558.0,513.0,39.2,24235.0,50.0,05762
1,ZCTA5 05763,54968.0,2715.0,43.9,29674.0,488.0,05763
2,ZCTA5 05764,44468.0,3370.0,41.3,22651.0,323.0,05764
3,ZCTA5 05765,56771.0,1632.0,46.7,30786.0,85.0,05765
4,ZCTA5 05766,52250.0,598.0,45.9,33227.0,76.0,05766
...,...,...,...,...,...,...,...
33115,ZCTA5 99518,82119.0,9872.0,36.4,43211.0,478.0,99518
33116,ZCTA5 99519,-666666666.0,542.0,51.5,121679.0,0.0,99519
33117,ZCTA5 99540,100755.0,286.0,62.3,50354.0,9.0,99540
33118,ZCTA5 99546,75000.0,122.0,29.9,24629.0,45.0,99546


In [7]:
# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"].astype(float) / census_pd["Population"].astype(float)
census_pd.head()

,Name,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Zipcode,Poverty Rate
0,ZCTA5 05762,58558.0,513.0,39.2,24235.0,50.0,05762,9.746589
1,ZCTA5 05763,54968.0,2715.0,43.9,29674.0,488.0,05763,17.974217
2,ZCTA5 05764,44468.0,3370.0,41.3,22651.0,323.0,05764,9.584570
3,ZCTA5 05765,56771.0,1632.0,46.7,30786.0,85.0,05765,5.208333
4,ZCTA5 05766,52250.0,598.0,45.9,33227.0,76.0,05766,12.709030


In [8]:
# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate"]]

census_pd.head()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,05762,513.0,39.2,58558.0,24235.0,50.0,9.746589
1,05763,2715.0,43.9,54968.0,29674.0,488.0,17.974217
2,05764,3370.0,41.3,44468.0,22651.0,323.0,9.584570
3,05765,1632.0,46.7,56771.0,30786.0,85.0,5.208333
4,05766,598.0,45.9,52250.0,33227.0,76.0,12.709030


In [9]:
# Visualize
print(len(census_pd))

33120


In [10]:
zip_codes=[90001, 90002, 90005, 90006, 90003, 90004, 90007, 90008, 90010, 90013, 90014, 90017, 90021, 90009, 90011, 90012, 90015, 90016,
90018, 90019, 90020, 90022, 90025, 90027, 90031, 90032, 90023, 90024, 90026, 90028, 90029, 90030, 90033, 90034, 90037, 90039, 90042, 90043, 90045, 90035, 90036, 90038, 90040, 90041, 90044, 90046, 90048, 90052, 90053, 90054, 90057, 90058, 90047, 90049, 90050, 90051, 90055, 90056, 90059, 90062, 90063, 90064, 90065, 90066, 90070, 90060, 90061, 90067, 90068, 90071, 90072, 90074, 90078, 90079, 90084, 90086, 90087, 90095, 90096, 90073, 90075, 90076, 90077, 90080, 90081, 90082, 90083, 90088,
90089, 90091, 90093, 90099, 90189, 90101, 90103]

zip_codes_df=pd.DataFrame({"Zipcode": zip_codes})
zip_codes_df.head()

,Zipcode
0,90001
1,90002
2,90005
3,90006
4,90003


In [11]:
zip_codes_df.count

<bound method DataFrame.count of     Zipcode
0     90001
1     90002
2     90005
3     90006
4     90003
..      ...
90    90093
91    90099
92    90189
93    90101
94    90103

[95 rows x 1 columns]>

In [12]:
census_pd.dtypes

Zipcode               object
Population           float64
Median Age           float64
Household Income     float64
Per Capita Income    float64
Poverty Count        float64
Poverty Rate         float64
dtype: object

In [13]:
census_pd['Zipcode'].astype(str)

0        05762
1        05763
2        05764
3        05765
4        05766
         ...  
33115    99518
33116    99519
33117    99540
33118    99546
33119    99547
Name: Zipcode, Length: 33120, dtype: object

In [14]:
zip_codes_df.dtypes

Zipcode    int64
dtype: object

In [15]:
zip_codes_df['Zipcode']=zip_codes_df['Zipcode'].astype(str)

In [16]:
zip_codes_df.dtypes

Zipcode    object
dtype: object

In [17]:
la_df=pd.merge(zip_codes_df, census_pd, on='Zipcode', how='inner')
la_df.head()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,90001,57942.0,28.0,34323.0,11294.0,18902.0,32.622277
1,90002,51826.0,27.5,32520.0,11212.0,18302.0,35.314321
2,90005,39338.0,33.9,31485.0,20265.0,11520.0,29.284661
3,90006,61230.0,33.6,31993.0,14306.0,17456.0,28.508901
4,90003,70208.0,28.0,31878.0,10611.0,24556.0,34.976071


In [18]:
la_df.count()

Zipcode              65
Population           65
Median Age           65
Household Income     65
Per Capita Income    65
Poverty Count        65
Poverty Rate         62
dtype: int64

In [19]:
la_df=la_df.dropna()
la_df.count()

Zipcode              62
Population           62
Median Age           62
Household Income     62
Per Capita Income    62
Poverty Count        62
Poverty Rate         62
dtype: int64

In [20]:
gmaps.configure(api_key=gkey)

In [21]:
pip install pgeocode

Note: you may need to restart the kernel to use updated packages.


In [22]:
import pgeocode

In [23]:
lat_lon = []

nomi = pgeocode.Nominatim('US')

for code in zip_codes:

    geo_metrics = nomi.query_postal_code(code)
    lat_lon.append(geo_metrics.to_dict())
    
lat_lon_df = pd.DataFrame(lat_lon)

lat_lon_df = lat_lon_df.rename(columns={"postal_code":"Zipcode",
                                       "latitude": "Latitude",
                                       "longitude": "Longitude"})
lat_lon_df = lat_lon_df[["Zipcode", "Latitude", "Longitude"]]
lat_lon_df = lat_lon_df.dropna()
lat_lon_df

,Zipcode,Latitude,Longitude
0,90001,33.9731,-118.2479
1,90002,33.9497,-118.2462
2,90005,34.0585,-118.3012
3,90006,34.0493,-118.2917
4,90003,33.9653,-118.2727
...,...,...,...
88,90089,33.7866,-118.2987
89,90091,34.0522,-118.2437
90,90093,34.0522,-118.2437
91,90099,34.0522,-118.2437


In [24]:
la_df=pd.merge(la_df, lat_lon_df, on='Zipcode', how='inner')
la_df.head(62)

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Latitude,Longitude
0,90001,57942.0,28.0,34323.0,11294.0,18902.0,32.622277,33.9731,-118.2479
1,90002,51826.0,27.5,32520.0,11212.0,18302.0,35.314321,33.9497,-118.2462
2,90005,39338.0,33.9,31485.0,20265.0,11520.0,29.284661,34.0585,-118.3012
3,90006,61230.0,33.6,31993.0,14306.0,17456.0,28.508901,34.0493,-118.2917
4,90003,70208.0,28.0,31878.0,10611.0,24556.0,34.976071,33.9653,-118.2727
...,...,...,...,...,...,...,...,...,...
57,90067,2355.0,65.8,120208.0,152254.0,176.0,7.473461,34.0551,-118.4095
58,90068,21450.0,39.7,76485.0,75182.0,2490.0,11.608392,34.1156,-118.3305
59,90073,915.0,54.3,-666666666.0,11281.0,439.0,47.978142,33.7866,-118.2987
60,90077,8912.0,47.9,178194.0,100874.0,311.0,3.489677,34.1112,-118.4502


In [25]:
# Store 'Lat' and 'Lng' into  locations 
locations = la_df[["Latitude", "Longitude"]].astype(float)

In [26]:
# Convert Poverty Rate to float and store
poverty_rate = la_df["Poverty Rate"].astype(float)

In [27]:
weights = la_df["Poverty Rate"].astype(float)
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=poverty_rate, dissipating=True, opacity=0.6))
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
la_df.to_csv("Output/2016.csv", index=False, header=True)